In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, precision_score
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
import time
from tqdm.notebook import tqdm
import torch.nn.functional as F
import sklearn


In [2]:
class ANN_Model(nn.Module):
    def __init__(self, 
                 input_features = 16, 
                 out_features=2,
                 hdn_dim=30):
       
        super().__init__()
       
        self.linear_relu_stack = nn.Sequential( 
            nn.Linear(input_features, hdn_dim),
            nn.Dropout(p=0.3),
            nn.ReLU(),
            nn.Linear(hdn_dim, hdn_dim),
            nn.Dropout(p=0.3),
            nn.ReLU(),
            nn.Linear(hdn_dim, hdn_dim),
            nn.Dropout(p=0.3),
            nn.ReLU(),
            nn.Linear(hdn_dim, hdn_dim),
            nn.Dropout(p=0.3),
            nn.ReLU(),
            nn.Linear(hdn_dim, out_features),
        )

    def forward(self, x):
        out = self.linear_relu_stack(x)
        return out

In [3]:
# Class to wrap a dataloader to move data to a device
class DeviceDataLoader(): 
    def __init__(self, dl, device):
        self.dl = dl   
        self.device = device
        
    def __iter__(self):
        for b in self.dl: 
            yield feed_device(b, self.device)
    
    def __len__(self):
        return len(self.dl)  


In [4]:
# Function to move data to the selected device
def feed_device(data, device): 
    if isinstance(data, (list, tuple)):
        return [feed_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [5]:
# Function to check device availability
def device_available(force_cpu=False): 
    if force_cpu or not torch.cuda.is_available():
        device = torch.device('cpu')
        print("Using CPU")
    else:
        device = torch.device('cuda')
        print("Using GPU")
    return device

In [6]:
def train_evaluate(model, train_loader, test_loader, device, loss_function, optimizer, epochs=20):
    model.train()
    for epoch in range(epochs):
        train_losses = [] #Store the losses of the model
        for inpt, target in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"): 
            inpt, target = inpt.to(device), target.to(device) #Move data to device
            optimizer.zero_grad() #clears gradients
            out=model(inpt) #FORWARD PASS: predictions
            loss = loss_function(out, target) #loss function: compares the model predictions to the real data.
            loss.backward()  
            optimizer.step()  
            train_losses.append(loss.item())
        avg_loss = np.mean(train_losses)
        print(f"Epoch {epoch+1}/{epochs}, Average Loss: {avg_loss:.4f}")
    #VALIDATION
    model.eval()
    test_preds, test_targets = [], []
    with torch.no_grad(): #to not calculate and update gradients since its a validation
        for inpt, target in test_loader:
            inpt, target = inpt.to(device), target.to(device)
            out = model(inpt)
            _, preds = torch.max(out, dim=1) 
            test_preds.extend(preds.cpu().numpy())
            test_targets.extend(target.cpu().numpy())
    
    #METRICS
    accuracy = accuracy_score(test_targets, test_preds)
    precision_pos = precision_score(test_targets, test_preds, pos_label=1)  
    precision_neg = precision_score(test_targets, test_preds, pos_label=0)
    return accuracy, precision_pos, precision_neg

In [7]:
def evaluate(model, test_loader, device):
    #VALIDATION
    model.eval()
    test_preds, test_targets = [], []
    with torch.no_grad(): #to not calculate and update gradients since its a validation
        for inpt, target in test_loader:
            inpt, target = inpt.to(device), target.to(device)
            out = model(inpt)
            _, preds = torch.max(out, dim=1) 
            test_preds.extend(preds.cpu().numpy())
            test_targets.extend(target.cpu().numpy())
    
    #METRICS
    accuracy = accuracy_score(test_targets, test_preds)
    precision_pos = precision_score(test_targets, test_preds, pos_label=1)  
    precision_neg = precision_score(test_targets, test_preds, pos_label=0)
    return accuracy, precision_pos, precision_neg

In [ ]:
# Main function to run the entire process
def main():
    # Load and prepare data
    df_sig = pd.read_csv("/home/emma-gutierrez/Servicio_Becario/HiggsBSM/higgsbsm-fw/hep_data/dataframe_electronesA0.csv")
    df_sig = df_sig.drop(['Z_boson_px_e'], axis=1)
    print(f"Length of df_sig: {len(df_sig)}")
    sig_class = np.full(int(len(df_sig.index)), 1)
    df_sig.insert(0, "class", sig_class, True)

    df_bkg = pd.read_csv("/home/emma-gutierrez/Servicio_Becario/HiggsBSM/higgsbsm-fw/hep_data/data_electrons_ttbar.csv")
    df_bkg = df_bkg.drop(["anti_bquark_e_e_bg"], axis=1)
    print(f"Length of df_bkg: {len(df_bkg)}")
    bkg_class = np.full(int(len(df_bkg.index)), 0)
    df_bkg.insert(0, "class", bkg_class, True)

    # Combine and sample data
    df_comb2 = pd.concat([df_sig, df_bkg])
    
    df_sig = df_comb2.loc[df_comb2['class'] == 1].sample(n=20000, random_state=None)
    df_bkg = df_comb2.loc[df_comb2['class'] == 0].sample(n=20000, random_state=None)
    df_comb = pd.concat([df_sig, df_bkg])
    df_comb = df_comb.sample(frac=1, random_state=1)

    print(df_comb.head())

    # Normalize data
    from sklearn.preprocessing import MinMaxScaler
    df_comb = df_comb.copy()
    df_comb = pd.DataFrame(MinMaxScaler().fit_transform(df_comb), columns=list(df_comb.columns))

    # Convert data to numpy arrays 
    X = df_comb.drop(['class'], axis=1).values.astype(np.float32)
    Y = df_comb['class'].values.astype(np.int64)

    # Split into train and test sets (80-20 split)
    from sklearn.model_selection import train_test_split
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

    
    np.save("X_test.npy", X_test)
    np.save("Y_test.npy", Y_test)

    # Convert to PyTorch tensors
    X_train = torch.tensor(X_train)
    Y_train = torch.tensor(Y_train)
    X_test = torch.tensor(X_test)
    Y_test = torch.tensor(Y_test)



    # Create DataLoaders
    batch_size = 4000
    train_dataset = TensorDataset(X_train, Y_train)
    test_dataset = TensorDataset(X_test, Y_test)
    
    train_loader = DataLoader(train_dataset, 
                            batch_size=64,
                            shuffle=True,
                            num_workers=4,
                            pin_memory=True)
    test_loader = DataLoader(test_dataset, 
                            batch_size=64,
                            shuffle=False,
                            num_workers=4,
                            pin_memory=True)

    # Initialize model and training components
    device = device_available(force_cpu=True)
    train_loader_cpu = DeviceDataLoader(train_loader, device)
    test_loader_cpu = DeviceDataLoader(test_loader, device)

    model = ANN_Model()
    model = feed_device(model, device)
    loss_function = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Train and evaluate
    start_time = time.time()
    accuracy, precision_pos, precision_neg = train_evaluate(model, train_loader_cpu, test_loader_cpu, device, loss_function, optimizer)
    training_time = time.time() - start_time

    # Store results
    results_cpu_df = pd.DataFrame({
        'Batch_Size': [batch_size],
        'Time': [training_time],
        'Accuracy': [accuracy],
        "Precision Pos": [precision_pos],
        "Precision Neg": [precision_neg]
    })

    print(f"CPU - Accuracy: {accuracy:.4f}, Precision Pos: {precision_pos:.4f}, Precision Neg: {precision_neg:.4f}, Training Time: {training_time:.2f}s")
    
    results_cpu_df.to_csv("results_cpu_nn.csv", index=False)

    return model, optimizer, test_loader_cpu

In [9]:
model, optimizer, test_loader_cpu = main()

Length of df_sig: 24949
Length of df_bkg: 24985
       class  electron_px  electron_py  electron_pz  electron_e  positron_px  \
40         1   -50.702323    78.444204  -111.100725  145.146787    28.634500   
24076      1    20.046127    50.883961   -86.161053  102.052691   -65.813686   
1843       1     6.555112   -12.950863   -66.652648   68.214880   -22.730597   
14479      0   -18.369341   -45.874699   -31.789330   58.757827   -61.805300   
19630      1     8.822884    89.847724    27.694229   94.432130   -41.767997   

       positron_py  positron_pz  positron_e  bquark_px_e  bquark_py_e  \
40       19.562514    -9.367205   35.921735   -22.718763   -70.033021   
24076    40.249762   -62.275120   99.144718    47.961157  -107.481970   
1843     23.861659    16.728138   36.957941    25.502742    -8.015626   
14479    54.096725  -193.577655  210.282332    18.001946   105.259459   
19630    -0.906520    16.076848   44.764410   -38.978515   -87.082532   

       bquark_pz_e  bquark_e_e  

Epoch 1/20:   0%|          | 0/500 [00:00<?, ?it/s]

Epoch 1/20, Average Loss: 0.6934


Epoch 2/20:   0%|          | 0/500 [00:00<?, ?it/s]

Epoch 2/20, Average Loss: 0.6934


...

Epoch 18/20, Average Loss: 0.6380


Epoch 19/20:   0%|          | 0/500 [00:00<?, ?it/s]

Epoch 19/20, Average Loss: 0.6301


Epoch 20/20:   0%|          | 0/500 [00:00<?, ?it/s]

Epoch 20/20, Average Loss: 0.6256
CPU - Accuracy: 0.6378, Precision Pos: 0.5966, Precision Neg: 0.7405, Training Time: 24.68s


In [10]:

# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

#Save model

torch.save(model.state_dict(), 'model_features.pth')



Model's state_dict:
linear_relu_stack.0.weight 	 torch.Size([30, 16])
linear_relu_stack.0.bias 	 torch.Size([30])
linear_relu_stack.3.weight 	 torch.Size([30, 30])
linear_relu_stack.3.bias 	 torch.Size([30])
linear_relu_stack.6.weight 	 torch.Size([30, 30])
linear_relu_stack.6.bias 	 torch.Size([30])
linear_relu_stack.9.weight 	 torch.Size([30, 30])
linear_relu_stack.9.bias 	 torch.Size([30])
linear_relu_stack.12.weight 	 torch.Size([2, 30])
linear_relu_stack.12.bias 	 torch.Size([2])
Optimizer's state_dict:
state 	 {0: {'step': tensor(10000.), 'exp_avg': tensor([[ 5.6052e-45,  5.6052e-45,  5.6052e-45,  5.6052e-45,  5.6052e-45,
          5.6052e-45,  5.6052e-45,  5.6052e-45,  5.6052e-45,  5.6052e-45,
          5.6052e-45,  5.6052e-45,  5.6052e-45,  5.6052e-45,  5.6052e-45,
          5.6052e-45],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+0

In [11]:

model = ANN_Model()

model.load_state_dict(torch.load('model_features.pth', weights_only=True))

device = device_available(force_cpu=True)



Using CPU


In [12]:
model.eval()  # This disables dropout/batch norm if you have any

# Run evaluation using your existing function
accuracy, precision_pos, precision_neg = evaluate(model, test_loader_cpu, device)

# Print results
print(f"Accuracy: {accuracy}")
print(f"Positive Precision: {precision_pos}")
print(f"Negative Precision: {precision_neg}")

Accuracy: 0.63775
Positive Precision: 0.5965680266153038
Negative Precision: 0.7404980340760158
